In [ ]:
"""


當用戶關注Line@後，Line會發一個FollowEvent，

我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳四個消息給用戶


"""

In [ ]:
"""

啟用伺服器基本樣板

"""

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [ ]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='如何建造公民電廠',
        text='這個小bot，可以做為你認識「公民/社區電廠」的入門指引， 並按部就班打造屬於你和夥伴的公民電廠。',
        actions= [
          {
            "type": "postback",
            "label": "流程",
#             "text": "切換圖文選單",
            "data": "menu=update"
          },
          {
            "type": "uri",
            "label": "範例",
            "uri": "https://www.facebook.com/notes/陳秉亨/公民綠能電廠實驗故事一達魯瑪克部落/10157810738229796/"
          },
            {
           "type": "uri",
            "label": "費用試算",
            "uri": "https://www.mrpv.org.tw/SettingFee/PubSettingfee.aspx?type=setup_info&post_id=60",
          },
          {
           "type": "postback",
            "label": "問卷",
            "data": "quick=quickreply"
          }
        ],           
)
)

In [ ]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)
#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)

# 消息清單
reply_message_list = [
    TextSendMessage(text="我的能源我主宰，社區就是發電廠。"),
    
    TextSendMessage(text="很多人每天下班前，已經想好晚餐吃什麼？預算多少?但每當要繳電費時，身為消費 者的我們，卻沒有任何選擇權。\n\n關於能源，我們不只是消費者，其實也可以是生產者，發電不只與政府、企業有關， 一般社區的你我，也有能力參與。"),
    
    ImageSendMessage(original_content_url='https://%s/images/1.jpg' %server_url ,
    preview_image_url='https://%s/images/1.jpg' %server_url),
#     ImageSendMessage(original_content_url='https://%s/images/004.png' %server_url,
#     preview_image_url='https://%s/images/005.jpg' %server_url)
    buttons_template_message
]


In [ ]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''
users_dict = {} #Shadow_add


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests

global user_name

user_name=[]

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    global user_name
    global users_dict #Shadow_add

    # 取出消息內User的資料
    # line get user profile python example    
    user_profile = line_bot_api.get_profile(event.source.user_id)
    user_name = user_profile.display_name #使用者名稱
    uid = user_profile.user_id # 發訊者ID
    
    # ======= Shadow_add =========
    if uid not in users_dict:
        users_dict[uid] = 0
    # ============================    
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkResult=line_bot_api.link_rich_menu_to_user(secretFileContentJson["self_user_id"], linkRichMenuId)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )


In [ ]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage,VideoSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    print("fileName:",fileName)
    with open(fileName,'r',encoding='utf8') as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    


    # 回傳
    return returnArray

In [ ]:
#用戶填寫問卷，對其回傳做相對應處理(儲存問卷答案)
a={}
b={}
from urllib.parse import parse_qs
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)# shadow_add
    uid = user_profile.user_id # shadow_add
    global a
    global users_dict
    if users_dict[uid]<7: # shadow_add
        if event.message.text ==u"A":
            a[users_dict[uid]-1] = event.message.text # shadow_add
            result_message_array =[] 
            replyJsonPath = './material/quickreply/'+f"reply{users_dict[uid]}.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            
            line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
            )
            
            users_dict[uid] += 1# shadow_add
            print("a:",a)
        elif event.message.text ==u"B": 
            a[users_dict[uid]-1] = event.message.text# shadow_add
#             a[users_dict[uid]-1]=event.message.text
            result_message_array =[]
            replyJsonPath = './material/quickreply/'+f"reply{users_dict[uid]}.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            
            line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
            )
            
            users_dict[uid] += 1# shadow_add
            print("a:",a)
        elif event.message.text ==u"C":
            a[users_dict[uid]-1] = event.message.text# shadow_add

            result_message_array =[]
            replyJsonPath = './material/quickreply/'+f"reply{users_dict[uid]}.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            
            line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
            )
            
            users_dict[uid] += 1# shadow_add
            print("a:",a)
        elif event.message.text ==u"D":
            a[users_dict[uid]-1] = event.message.text# shadow_add

            result_message_array =[]
            replyJsonPath = './material/quickreply/'+f"reply{users_dict[uid]}.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            
            line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
            )

            users_dict[uid] += 1# shadow_add
            print("users_dict[uid]=",users_dict[uid])
            print("a:",a)
        elif event.message.text !=u"A" or u"B" or u"C" or u"D":
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("你填錯囉，再自己填一次喔")
            )
    
    elif users_dict[uid]==7:
#         a[i-1] = event.message.text
        a[users_dict[uid]-1] = event.message.text# shadow_add
        b[uid] = a
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage("感謝填寫本次問卷")
        )

        print("a:",a)
        print("b", b)
        linksql(uid)
    else:
        line_bot_api.reply_message(event.reply_token,
        TextSendMessage(text=event.message.text))
    
@app.route('/')
def index():
    return 'Hello World'

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
from urllib.parse import parse_qs 
@handler.add(PostbackEvent)
def handle_post_message(event):
    global users_dict
    user_profile = line_bot_api.get_profile(event.source.user_id)
    query_string_dict = parse_qs(event.postback.data)
    uid = user_profile.user_id # shadow_add
    
    if 'menu' in query_string_dict:
        rich_menu_path='./material/'+query_string_dict.get('menu')[0]+'/rich_menu_id'
        print(rich_menu_path)
        
        linkRichMenuId = open(rich_menu_path, 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        

    elif 'folder' in query_string_dict:
        result_message_array =[]
        
        replyJsonPath = './material/'+query_string_dict.get('folder')[0]+'/reply.json'
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

    elif 'data' in query_string_dict:
        line_bot_api.reply_message(
            event.reply_token,
            buttons_template_message
        )
        
    elif 'quick' in query_string_dict:
        result_message_array =[]
        replyJsonPath = './material/'+query_string_dict.get('quick')[0]+'/'+f"reply{users_dict[uid]}.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        users_dict[uid] += 1 #shadow_add
    else:
        pass

In [ ]:
# 將使用者資料及問卷答案儲存於資料庫中

def linksql(uid):
    global user_name
    global a, b
    
    import pymysql
    import pymysql.cursors

    link=pymysql.connect(
    host="localhost",
    user="root",
    passwd="",
    db="test",
    charset="utf8"
    )
    c=link.cursor()

    param = {
        "user_name":user_name,
        "Q1":b[uid][0],
        "Q2":b[uid][1],
        "Q3":b[uid][2],
        "Q4":b[uid][3],
        "Q5":b[uid][4],
        "Q6":b[uid][5],
        "Q7":b[uid][6]
    }
    c.execute('''INSERT INTO `Questionnaire`(user_name,Q1,Q2,Q3,Q4,Q5,Q6,Q7
    ) VALUES(%(user_name)s,%(Q1)s,%(Q2)s,%(Q3)s,%(Q4)s,%(Q5)s,%(Q6)s,%(Q7)s)''',param)
    link.commit()
    link.close()

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

# if __name__ == "__main__":
#     app.run(host='0.0.0.0') #, debug=True

In [ ]:
'''

Application 運行（heroku版）

'''

import os
if __name__ == "__main__":
    app.run(host='0.0.0.0',port=os.environ['PORT'])